## FoundryIQ Creation

In [2]:
pip install azure-search-documents==11.7.0b2


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\lananoor\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import os
from dotenv import load_dotenv


In [3]:
# Check latest Azure AI Search SDK Version (11.7.0b2)
import azure.search.documents
print(azure.search.documents.__version__)


11.7.0b2


In [ ]:
# Authenticate using AAIS key
from azure.core.credentials import AzureKeyCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from dotenv import load_dotenv

load_dotenv()

index_client = SearchIndexClient(endpoint = os.getenv("AAIS_ENDPOINT"), credential = AzureKeyCredential(os.getenv("AAIS_KEY")))

### Azure AI Search Index - Knowledge Source #1

In [36]:
# Create a search index knowledge source
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters, SearchIndexFieldReference

knowledge_source = SearchIndexKnowledgeSource(
    name = "housing-index-ks",
    description= "Use this when user query relates to policy analysis, market outlooks, risk monitoring, housing finance insights, and regulatory or institutional reporting that require combining structural housing market context with operational mortgage performance and borrower behavior data.",
    encryption_key = None,
    search_index_parameters = SearchIndexKnowledgeSourceParameters(
        search_index_name = "housing-demo-41-index",
        semantic_configuration_name = "my-semantic-config",
        source_data_fields = [
            SearchIndexFieldReference(name="text"),
            SearchIndexFieldReference(name="custom_page_processing_steps"),
        ],
        search_fields = [
            SearchIndexFieldReference(name="page_image_path")
        ],
    )
)

index_client.create_or_update_knowledge_source(knowledge_source)
print(f"Knowledge source '{knowledge_source.name}' created or updated successfully.")

Knowledge source 'housing-index-ks' created or updated successfully.


## Azure AI Search - Knowledge Source #2

In [35]:
# Create a search index knowledge source
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters, SearchIndexFieldReference

knowledge_source = SearchIndexKnowledgeSource(
    name = "handwritten-index-ks",
    description= "Use this when user query relates to purchase agreements, resale contracts, and new home or construction contracts, with manually filled buyer/seller details, property information, pricing, financing terms, earnest money, disclosures, addenda, and closing conditions.",
    encryption_key = None,
    search_index_parameters = SearchIndexKnowledgeSourceParameters(
        search_index_name = "handwritten-demo-41-index",
        semantic_configuration_name = "my-semantic-config",
        source_data_fields = [
            SearchIndexFieldReference(name="text"),
            SearchIndexFieldReference(name="custom_page_processing_steps"),
        ],
        search_fields = [
            SearchIndexFieldReference(name="page_image_path")
        ],
    )
)

index_client.create_or_update_knowledge_source(knowledge_source)
print(f"Knowledge source '{knowledge_source.name}' created or updated successfully.")

Knowledge source 'handwritten-index-ks' created or updated successfully.


### Azure Blob Storage - Knowledge Source #3 

In [ ]:
# Azure Blob knowledge source for housing docs
load_dotenv()

storage_connection_string = os.getenv("AZURE_STORAGE_ACCOUNT_CONNECTION_STRING")
if not storage_connection_string:
    raise ValueError("AZURE_STORAGE_ACCOUNT_CONNECTION_STRING is not set.")

aoai_endpoint = os.getenv(
    "AZURE_OPENAI_ENDPOINT"
)
aoai_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT", "gpt-5-mini")
aoai_embedding_deployment = os.getenv(
    "AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-small"
)

chat_model = KnowledgeBaseAzureOpenAIModel(
    azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
        resource_url=aoai_endpoint,
        deployment_name=aoai_chat_deployment,
        model_name=aoai_chat_deployment,
    )
)

embedding_model = KnowledgeSourceAzureOpenAIVectorizer(
    azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
        resource_url=aoai_endpoint,
        deployment_name=aoai_embedding_deployment,
        model_name=aoai_embedding_deployment,
    )
)

knowledge_source = AzureBlobKnowledgeSource(
    name="products-blob-ks",
    description=(
        "Use this when query relates to product catalogs, specifications, pricing details, and inventory information across various categories, brands, and suppliers."
    ),
    azure_blob_parameters=AzureBlobKnowledgeSourceParameters(
        connection_string=storage_connection_string,
        container_name="products-container",
        folder_path=None,
        is_adls_gen2=False,
        ingestion_parameters=KnowledgeSourceIngestionParameters(
            disable_image_verbalization=False,
            chat_completion_model=chat_model,
            embedding_model=embedding_model,
            content_extraction_mode=KnowledgeSourceContentExtractionMode.MINIMAL,
        ),
    ),
)

index_client.create_or_update_knowledge_source(knowledge_source)
print(f"Knowledge source '{knowledge_source.name}' created or updated successfully.")

Knowledge source 'products-blob-ks' created or updated successfully.


### Azure Blob Storage - Knowledge Source #4

In [ ]:
from azure.search.documents.indexes.models import (
    AzureBlobKnowledgeSource,
    AzureBlobKnowledgeSourceParameters,
    KnowledgeSourceIngestionParameters,
    KnowledgeSourceContentExtractionMode,
    AIServices,
)

ai_services = AIServices(
    # IMPORTANT: this is an ENDPOINT/URI, not a resourceId
    uri=PROJECT_SUB_AI_SERVICE,   
    api_key=PROJECT_API_KEY,
)

knowledge_source = AzureBlobKnowledgeSource(
    name="housing-blob-cu-ks",
    description="Use this when query relates to housing market data…",
    azure_blob_parameters=AzureBlobKnowledgeSourceParameters(
        connection_string=storage_connection_string,
        container_name="housing-container",
        ingestion_parameters=KnowledgeSourceIngestionParameters(
            disable_image_verbalization=False,
            chat_completion_model=chat_model,
            embedding_model=embedding_model,
            content_extraction_mode=KnowledgeSourceContentExtractionMode.STANDARD, # Content Understanding Ingestion due to multimodal document
            ai_services=ai_services,
        ),
    ),
)

index_client.create_or_update_knowledge_source(knowledge_source)
print(f"Knowledge source '{knowledge_source.name}' created or updated successfully.")

Knowledge source 'housing-blob-cu-ks' created or updated successfully.


In [65]:
load_dotenv()

storage_connection_string = os.getenv("AZURE_STORAGE_ACCOUNT_CONNECTION_STRING")
if not storage_connection_string:
    raise ValueError("AZURE_STORAGE_ACCOUNT_CONNECTION_STRING is not set.")

aoai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "https://agents-resource-foundry-resource.openai.azure.com/")

# Use a chat deployment that supports max_tokens (e.g., gpt-4o-mini)
aoai_ingestion_chat_deployment = os.getenv("AZURE_OPENAI_INGESTION_CHAT_DEPLOYMENT", "gpt-4o")
aoai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")

chat_model_for_ingestion = KnowledgeBaseAzureOpenAIModel(
    azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
        resource_url=aoai_endpoint,
        deployment_name=aoai_ingestion_chat_deployment,
        model_name=aoai_ingestion_chat_deployment,
    )
)

embedding_model = KnowledgeSourceAzureOpenAIVectorizer(
    azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
        resource_url=aoai_endpoint,
        deployment_name=aoai_embedding_deployment,
        model_name=aoai_embedding_deployment,
    )
)

ai_services = AIServices(
    uri=PROJECT_SUB_AI_SERVICE,
    api_key=PROJECT_API_KEY,
)

knowledge_source = AzureBlobKnowledgeSource(
    name="housing-blob-cu-ks-v4",
    description="Use this when query relates to housing market data…",
    azure_blob_parameters=AzureBlobKnowledgeSourceParameters(
        connection_string=storage_connection_string,
        container_name="housing-container",
        ingestion_parameters=KnowledgeSourceIngestionParameters(
            disable_image_verbalization=False,
            chat_completion_model=chat_model_for_ingestion,
            embedding_model=embedding_model,
            content_extraction_mode=KnowledgeSourceContentExtractionMode.STANDARD,
            ai_services=ai_services,
        ),
    ),
)

index_client.create_or_update_knowledge_source(knowledge_source)
print(f"Knowledge source '{knowledge_source.name}' created or updated successfully.")

Knowledge source 'housing-blob-cu-ks-v4' created or updated successfully.


## Knowledge Base

In [ ]:
# Create a knowledge base
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeSourceReference, AzureOpenAIVectorizerParameters, KnowledgeRetrievalOutputMode, KnowledgeRetrievalLowReasoningEffort

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url = os.getenv("PROJECT_ENDPOINT"),,
    deployment_name = "gpt-5-mini",
    model_name = "gpt-5-mini",
)

knowledge_base = KnowledgeBase(
    name = "demo-full-kb", # pick name 
    description = "This knowledge base handles questions about organizational housing and mortgage finance reports, including affordable housing activities, housing goals, and market performance.",
    retrieval_instructions = "Information is contained across annual housing reports. Use the relevant report year when answering questions related to housing goals, mortgage market data, affordable housing initiatives, or Fannie Mae and Freddie Mac performance.",
    answer_instructions = "Provide a clear and concise answer grounded strictly in the retrieved report content.",
    output_mode = KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS,
    knowledge_sources = [
        KnowledgeSourceReference(name = "housing-index-ks"),
        KnowledgeSourceReference(name = "handwritten-index-ks"),
        KnowledgeSourceReference(name = "products-blob-ks"), 
        KnowledgeSourceReference(name = "housing-blob-ks"),
    ],
    models = [KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters = aoai_params)],
    encryption_key = None,
    retrieval_reasoning_effort = KnowledgeRetrievalLowReasoningEffort,
)

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base.name}' created or updated successfully.")

Knowledge base 'demo-full-kb' created or updated successfully.
